<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/custom_entity_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
ls -l

total 101276
-rw------- 1 root root     2000 Dec 26 18:06  bread_keyword_patterns.npz
-rw------- 1 root root      589 Dec 26 17:27  breads_list_2.jsonl
-rw------- 1 root root    26438 Dec 26 16:44  breads_list.jsonl
-rw------- 1 root root    25359 Dec 15 03:21  CoherenceScore.csv
-rw------- 1 root root     4968 Dec 26 18:06  condiment_keyword_patterns.npz
-rw------- 1 root root     2353 Dec 26 16:29  condiments_list_2.jsonl
-rw------- 1 root root    43320 Dec 26 16:21  condiments_list.jsonl
-rw------- 1 root root  8428475 Dec 21 13:01  datalist_corrected.csv
-rw------- 1 root root    24864 Dec 26 17:05  df_check_breads.csv
-rw------- 1 root root    40909 Dec 26 16:21  df_check_condiments.csv
-rw------- 1 root root   185178 Dec 26 08:09  df_check_de.csv
-rw------- 1 root root  4580904 Dec 26 08:10  df_check_en.csv
-rw------- 1 root root    56341 Dec 26 09:17  df_check_es.csv
-rw------- 1 root root    43595 Dec 26 09:17  df_check_fr.csv
-rw------- 1 root root    72573 Dec 26 17:05  df_ch

In [0]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

nlp = English()
ruler = EntityRuler(nlp).from_disk("patterns.jsonl")

In [0]:
from spacy import displacy
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML

In [5]:
ruler.patterns[10]

{'id': '5_hour_energy',
 'label': 'dish',
 'pattern': [{'LOWER': '5'}, {'LOWER': 'hour'}, {'LOWER': 'energy'}]}

In [0]:
nlp.add_pipe(ruler)

In [9]:
ruler.patterns[6200]

{'id': 'litsea_cubeba',
 'label': 'spice',
 'pattern': [{'LOWER': 'litsea'}, {'LOWER': 'cubeba'}]}

In [25]:
doc1 = nlp("I love fettuccine cheese but hate buffalo wings.")
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc1.ents])

displacy.render(doc1,'ent',jupyter=True)

doc2 = nlp("Acocados are wonderful but the ajwain is great")
displacy.render(doc2,'ent',jupyter=True)


[('fettuccine', 'dish', 'fettuccine'), ('buffalo wings', 'dish', 'buffalo_wings')]


In [13]:
reviews = pd.read_csv('reviews_text.csv',usecols=['_id', 'text'])
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 2 columns):
_id     56156 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 877.6+ KB


In [14]:
reviews.dropna(inplace=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56155 entries, 0 to 56155
Data columns (total 2 columns):
_id     56155 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 1.3+ MB


In [15]:
texts = reviews['text']
texts

0        Burger joint offers a wide range of cheeseburg...
1        It was really good. The mushroom broth was esp...
2        $8.90 for sesame rice, mushroom rendang, curry...
3        2 mains + 1 green rice bento from greendot. Re...
4        The lion mane mushroom rendang is so delicious...
                               ...                        
56151    Good love office treats! These are perfectly s...
56152    Very tasty! Crisp fresh vegetables with fried ...
56153    Delicious! Creamy coconut milk red curry sauce...
56154    Absolutely delicious! Great if you’re on the r...
56155    (Added the vegan coco whip on top at home)\nGo...
Name: text, Length: 56155, dtype: object

In [0]:
ingredients_list = ['pistachio', 'almond', 'walnut', 'cashew', 'peanuts', 'macadamia', 'hazelnuts', 'pecan', 'brazil nut', 'pine nut' ]
issues_list = ['crumble - maybe a verb', 'milk']
food_times = 
to_be_added_list = ['spirulina', 'herbs', 'micro-greens', 'greens','spinach','eggs', 'nut butters', 'cream cheese', 'cake', 'pepper', 'zucchini', 'aubergines',  'tomato',\
                    'oysters', 'kale', 'cucumber', 'quinoa', 'tomatoes', 'onion' ,'garlic', 'avocado', 'chilli', 'dark chocolate', 'mushrooms', 'coriander' ,'corn chips',\
                    'marinara', 'donuts', 'olives', 'crackers', 'waffles', 'strawberry', 'blueberry', 'raspberry', 'compote', 'mayo', 'mayonnaise', 'soy milk', 'oat milk', \
                    'coconut milk', 'almond milk', 'caffeine', 'milk tea', 'cauliflower', 'matcha', 'sorbet', 'banana', 'goji berry', 'chickpea', 'spinach', 'romaine',\
                    'arugula', 'cacao', 'Portobello', 'pear', 'plant milk', 'BBQ sauce', 'artichoke', 'mango', 'sandwich', 'capsicum', 'wedge', 'kang kong', 'seitan',\
                    'ciabatta', 'focaccia']

In [34]:
check = np.random.randint(0,50000,10)
print(check)

for doc in list(nlp.pipe(texts[check])):
  displacy.render(doc,'ent',jupyter=True)
  displacy.render(doc,'dep',jupyter=True)



[32173 26770 42691  7738 10876  7637 44355 12163 28296  6900]


In [35]:

displacy.render(nlp("Very fresh and juicy coconut FRUIT Lassi DISH . Didn't thought it would be so fresh"),'dep',jupyter=True)